[tutorial del uso de automodel for causal]('https://medium.com/@marketing_novita.ai/mastering-automodelforcausallm-a-handbook-for-novices-88ebdbad9736')

[tutorial del uso de mistral como modelo](https://www.datacamp.com/tutorial/mistral-7b-tutorial)

[access token huggingface](https://huggingface.co/settings/tokens)

[memory usage](C:\Users\byacu\.cache\huggingface\hub)

In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from dotenv import load_dotenv
import time
print(torch.cuda.is_available())  # Esto debe devolver True si tienes una GPU disponible
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

d:\Proyectos\Docker\Multimodal_RAG\amb_mm_rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


True

In [2]:
# Leer el token desde las variables de entorno
token = os.getenv("Mistral_access")
# Verifica que el token se haya cargado correctamente
if not token:
    raise ValueError("No se pudo cargar el token MISTRAL_ACCESS_TOKEN. Asegúrate de que esté configurado correctamente en el archivo .env.")

In [3]:
# Cargar el tokenizador y el modelo Mistral 7B
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=token)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.53s/it]


In [4]:
# Verificar si el tokenizador tiene un pad_token, si no, asignar el eos_token como pad_token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [5]:
# Verificar si la GPU está disponible y mover el modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
     

In [5]:
def generate_response(input_text):
    # Tokenizar el input y mover los datos a la GPU
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generar respuesta
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=150, do_sample=True)

    # Decodificar la respuesta
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [6]:
def generate_response(input_text):
    # Tokenizar el input, agregando padding y attention_mask
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    inputs = inputs.to(device)

    # Generar respuesta
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],  # Añadir attention mask
            max_length=150,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id  # Asegurarnos de que se utiliza el pad_token_id
        )

    # Decodificar la respuesta
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [7]:
%%time
print("Bienvenido al chat con Mistral 7B. Escribe algo para empezar (escribe 'salir' para terminar):")
user_input = input("Tú: ")
if user_input.lower() == "salir":
    print("¡Adiós!")


# Generar y mostrar la respuesta del modelo
response = generate_response(user_input)
print(f"Mistral: {response}\n")

Bienvenido al chat con Mistral 7B. Escribe algo para empezar (escribe 'salir' para terminar):


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Mistral: Hi, how are you? Yesterday I was in another country and I’m now here. This was one of the 3 biggest surprises of last night, the other 2 also have to do with a country. 😀

I had an excellent, relaxed and extremely productive weekend. And to finish it perfectly, I could watch a movie I really like, so I got on Netflix in the morning and started searching for “The Grand Budapest Hotel”.

When it finally appeared on my screen, I pressed play and… oh my goodness, I was in Austria! The weather is the same as the one from “The Grand Budapest Hotel”. 😀

Now I know that I’ll only be here 3

CPU times: total: 35 s
Wall time: 1min 40s


## DistilGPT-2

In [15]:
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=token)

d:\Proyectos\Docker\Multimodal_RAG\amb_mm_rag\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\byacu\.cache\huggingface\hub\models--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [16]:
# Verificar si la GPU está disponible y mover el modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [18]:
def generate_response(input_text):
    # Tokenizar el input y moverlo a la GPU
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Medir el tiempo de inferencia
    start_time = time.time()

    # Generar la respuesta
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=150, 
            do_sample=True, 
            pad_token_id=tokenizer.eos_token_id  # Evitar advertencias
        )

    # Medir el tiempo final
    inference_time = time.time() - start_time

    # Decodificar la respuesta
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response, inference_time

In [23]:
print("Bienvenido al chat con DistilGPT-2. Escribe algo para empezar (escribe 'salir' para terminar):")


user_input = input("Tú: ")
if user_input.lower() == "salir":
    print("¡Adiós!")

# Generar y mostrar la respuesta del modelo
response, time_taken = generate_response(user_input)
print(f"DistilGPT-2: {response}")
print(f"Tiempo de inferencia: {time_taken:.4f} segundos\n")

Bienvenido al chat con DistilGPT-2. Escribe algo para empezar (escribe 'salir' para terminar):
DistilGPT-2: Que haces? What were the worst fears on the face of the world. What did they end up doing?




“The first three months of June are the worst yet, according to this report by the UN and US Congress’ . In their response to the ongoing refugee crisis we’re dealing with the humanitarian crisis.
“The UN is demanding a resolution that is consistent with the UN Refugee Convention which was signed in the wake of the refugee crisis, and is calling for a resumption of the process."
“The United Nations estimates that every 1 million asylum seeker resettled would be internally displaced or those who are forced to leave their country.
“There are many ways of doing
Tiempo de inferencia: 2.8726 segundos



## GPT-fine tunned

In [24]:
# Cargar el tokenizador y el modelo GPT-2 ajustado para español
model_name = "datificate/gpt2-small-spanish"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

d:\Proyectos\Docker\Multimodal_RAG\amb_mm_rag\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\byacu\.cache\huggingface\hub\models--datificate--gpt2-small-spanish. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [25]:
# Verificar si la GPU está disponible y mover el modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [26]:
def generate_response(input_text):
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(inputs['input_ids'], max_length=150, do_sample=True)
    
    response_time = time.time() - start_time
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response, response_time

In [29]:
print("Bienvenido al chat en español. Escribe algo para empezar (escribe 'salir' para terminar):")

user_input = input("Tú: ")
if user_input.lower() == "salir":
    print("¡Adiós!")

response, time_taken = generate_response(user_input)
print(f"GPT-2 (Español): {response}")
print(f"Tiempo de inferencia: {time_taken:.4f} segundos\n")

Bienvenido al chat en español. Escribe algo para empezar (escribe 'salir' para terminar):


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GPT-2 (Español): ¿Cuáles son las mejores prestaciones paraempleados que debes dar en tu PyME? En tu sistema de correo electrónico hay que pedir al tu PC que tengate y que pague una máquina a tu PC.

Existen modelos de PC a tamaño entero. Algunos se encuentran en los últimos años.

La última serie de PC fue hecha para usuarios de PC y por ello han nacido los PC's de hasta para ser más grandes para ellos. Existen algunas variantes y PC's de hasta 10 MB.

Algunos de los títulos que más resaltan son: 


En esta serie los títulos en 2 partes fueron lanzados al mercado con PC el año 2004. El primero de este fue el mismo PC llamado K-21 (
Tiempo de inferencia: 3.7642 segundos

